In [ ]:
import zipfile
import os
import numpy as np
import tensorflow as tf
import io
import binvox_rw
import cv2
import matplotlib.pyplot as plt
from pathlib import Path
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import Sequence
from tensorflow.keras.layers import Input, Conv2D, Conv3DTranspose, Concatenate
from tensorflow.keras.layers import MaxPool2D, UpSampling3D, Reshape
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.losses import Loss

In [ ]:
def center(voxin):
    f = np.squeeze(np.max(voxin, axis=(2, 1)))
    s = np.squeeze(np.max(voxin, axis=(2, 0)))
    t = np.squeeze(np.max(voxin, axis=(1, 0)))

    nzf = np.sum(f == 0) 
    nzs = np.sum(s == 0)
    nzt = np.sum(t == 0)

    i_f = np.argmax(f)
    l_f = np.sum(f > 0)
    i_s = np.argmax(s)
    l_s = np.sum(s > 0)
    i_t = np.argmax(t)
    l_t = np.sum(t > 0)

    zfl = np.floor(nzf/2.0).astype(int)
    zfr = nzf - zfl
    zsl = np.floor(nzs/2.0).astype(int)
    zsr = nzs - zsl
    ztl = np.floor(nzt/2.0).astype(int)
    ztr = nzt - ztl  

    voxout = np.zeros_like(voxin)
    voxout[zfl:128-zfr, zsl:128-zsr, ztl:128-ztr] = voxin[i_f:i_f+l_f, i_s:i_s+l_s, i_t:i_t+l_t,]
    return voxout

In [ ]:
class voxel_gen(Sequence):
    def __init__(self, x_set, batch_size=32, dim=(128,128,128)):
        self.x = x_set # path for each dataset : models/models-binvox-solid
        self.batch_size = batch_size
        self.dim = dim

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.x) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        x_set_temp = self.x[index*self.batch_size:(index+1)*self.batch_size]
        X = self.__data_generation(x_set_temp)
        return X
    
    def __data_generation(self, x_set_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, 1))

        for i, x_set in enumerate(x_set_temp):
            # Store sample
            with open(str(x_set), 'rb') as f:
                model = binvox_rw.read_as_3d_array(f)
            X[i,] = np.expand_dims(center(model.data), axis=-1)
        return X

In [ ]:
def customGen(batch_size=32):
    gen = ImageDataGenerator(rescale=1/255.)
    view0 = gen.flow_from_directory(
        "models/models-screenshots/view0",
        target_size=(512, 512),
        shuffle=False,
        batch_size=batch_size,
        class_mode=None)

    view1 = gen.flow_from_directory(
        "models/models-screenshots/view1",
        target_size=(512, 512),
        shuffle=False,
        batch_size=batch_size,
        class_mode=None)

    view2 = gen.flow_from_directory(
        "models/models-screenshots/view2",
        target_size=(512, 512),
        shuffle=False,
        batch_size=batch_size,
        class_mode=None)

    view3 = gen.flow_from_directory(
        "models/models-screenshots/view3",
        target_size=(512, 512),
        shuffle=False,
        batch_size=batch_size,
        class_mode=None)
    
    view4 = gen.flow_from_directory(
        "models/models-screenshots/view4",
        target_size=(512, 512),
        shuffle=False,
        batch_size=batch_size,
        class_mode=None)
    
    view5 = gen.flow_from_directory(
        "models/models-screenshots/view5",
        target_size=(512, 512),
        shuffle=False,
        batch_size=batch_size,
        class_mode=None)
    
    view6 = gen.flow_from_directory(
        "models/models-screenshots/view6",
        target_size=(512, 512),
        shuffle=False,
        batch_size=batch_size,
        class_mode=None)
    
    view7 = gen.flow_from_directory(
        "models/models-screenshots/view7",
        target_size=(512, 512),
        shuffle=False,
        batch_size=batch_size,
        class_mode=None)
    
    view8 = gen.flow_from_directory(
        "models/models-screenshots/view8",
        target_size=(512, 512),
        shuffle=False,
        batch_size=batch_size,
        class_mode=None)
    
    view9 = gen.flow_from_directory(
        "models/models-screenshots/view9",
        target_size=(512, 512),
        shuffle=False,
        batch_size=batch_size,
        class_mode=None)
    
    view10 = gen.flow_from_directory(
        "models/models-screenshots/view10",
        target_size=(512, 512),
        shuffle=False,
        batch_size=batch_size,
        class_mode=None)
    
    view11 = gen.flow_from_directory(
        "models/models-screenshots/view11",
        target_size=(512, 512),
        shuffle=False,
        batch_size=batch_size,
        class_mode=None)
    
    view12 = gen.flow_from_directory(
        "models/models-screenshots/view12",
        target_size=(512, 512),
        shuffle=False,
        batch_size=batch_size,
        class_mode=None)
    
    view13 = gen.flow_from_directory(
        "models/models-screenshots/view13",
        target_size=(512, 512),
        shuffle=False,
        batch_size=batch_size,
        class_mode=None)
    
    
    voxels = Path("models/models-binvox-solid/data")
    fvoxels = [f for f in voxels.iterdir() if f.is_file()]
    
    out = voxel_gen(fvoxels, batch_size)
    while True:
        for x1, x2, x3, x4,x5, x6, x7, x8,x9, x10, x11, x12,x13,x14, y in zip(view0, view1, view2, view3,view4, view5, view6, view7,view8, view9, view10, view11,view12, view13,out):
            yield [x1, x2, x3, x4,x5, x6, x7, x8,x9, x10, x11, x12,x13,x14], y

In [ ]:
def encode_decode():
    inview0 = Input(shape=(512, 512, 3), name='view0')
    inview1 = Input(shape=(512, 512, 3), name='view1')
    inview2 = Input(shape=(512, 512, 3), name='view2')
    inview3 = Input(shape=(512, 512, 3), name='view3')
    inview4 = Input(shape=(512, 512, 3), name='view4')
    inview5 = Input(shape=(512, 512, 3), name='view5')
    inview6 = Input(shape=(512, 512, 3), name='view6')
    inview7 = Input(shape=(512, 512, 3), name='view7')
    inview8 = Input(shape=(512, 512, 3), name='view8')
    inview9 = Input(shape=(512, 512, 3), name='view9')
    inview10 = Input(shape=(512, 512, 3), name='view10')
    inview11 = Input(shape=(512, 512, 3), name='view11')
    inview12 = Input(shape=(512, 512, 3), name='view12')
    inview13 = Input(shape=(512, 512, 3), name='view13')
   
    encoder = Sequential([
            Conv2D(32, 7, (2, 2), activation='relu'),
            MaxPool2D((2, 2)),
            Conv2D(32, 5, activation='relu'),
            MaxPool2D((2, 2)),
            Conv2D(64, 3, activation='relu'),
            MaxPool2D((2, 2)),
            Conv2D(64, 3, activation='relu'),
            MaxPool2D((2, 2)),
            Conv2D(128, 3, activation='relu'),
            MaxPool2D((2, 2)),
            Conv2D(32, 5, activation='relu'),
            Reshape((1, 1, 1, -1))
    ], name='encoder')
    
    combined = Concatenate(axis=4)([encoder(inview0), encoder(inview1), encoder(inview2), encoder(inview3), encoder(inview4), encoder(inview5), encoder(inview6), encoder(inview7),encoder(inview8), encoder(inview9), encoder(inview10), encoder(inview11),encoder(inview12), encoder(inview13)])
    
    decoder = Sequential([
            Conv3DTranspose(128, 3, activation='relu'),
            UpSampling3D(2),
            Conv3DTranspose(64, 3, activation='relu'),
            UpSampling3D(2),
            Conv3DTranspose(32, 3, activation='relu', padding='same'),
            UpSampling3D(2),
            Conv3DTranspose(16, 3, activation='relu', padding='same'),
            UpSampling3D(2),
            Conv3DTranspose(8, 3, activation='relu', padding='same'),
            UpSampling3D(2)
    ], name='decoder')(combined)
    
    out = Conv3DTranspose(1, 3, activation='sigmoid', padding='same')(decoder)
    
    return Model(inputs=[inview0, inview1, inview2, inview3,inview4, inview5, inview6, inview7,inview8, inview9, inview10, inview11,inview12, inview13], outputs=out)

In [ ]:
tf.keras.backend.clear_session()
model = encode_decode()
model.summary(line_length=118, positions=[.38, .66, .75, 1.], expand_nested=True)

In [ ]:
class MSFCE(Loss):
    
    def call(self, y_true, y_pred):
        y_pred = tf.reshape(tf.convert_to_tensor(y_pred), [-1])
        y_true = tf.reshape(tf.cast(y_true, y_pred.dtype), [-1])
        P = tf.reduce_sum(y_true)
        N = 128**3 - P
        Pmask = tf.math.equal(y_true, tf.constant(1, dtype=tf.float32))
        Nmask = tf.math.equal(y_true, tf.constant(0, dtype=tf.float32))
        Vp = tf.boolean_mask(y_true, Pmask)
        Vp_pred = tf.boolean_mask(y_pred, Pmask)
        Vn = tf.boolean_mask(y_true, Nmask)
        Vn_pred = tf.boolean_mask(y_pred, Nmask)
        FNCE = -tf.reduce_mean(Vp * tf.math.log(tf.clip_by_value(Vp_pred, 1e-6, 1.)) + (1. - Vp) * tf.math.log(tf.clip_by_value(Vp_pred, 1e-6, 1.)))
        FPCE = -tf.reduce_mean(Vn * tf.math.log(tf.clip_by_value(Vn_pred, 1e-6, 1.)) + (1. - Vn) * tf.math.log(tf.clip_by_value(Vn_pred, 1e-6, 1.)))
        return FPCE + FNCE

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
loss = MSFCE()

In [ ]:
model.compile(optimizer, loss, metrics=[tf.keras.metrics.CosineSimilarity(), tf.keras.metrics.BinaryIoU()])

In [ ]:
model.fit(customGen(batch_size=2),epochs=5,steps_per_epoch=tf.math.ceil(11694/2))